In [1]:
import tweepy 
import pandas as pd
import time

In [2]:
# Twitter Developers에서 key를 발급받고 키 입력
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

# 개인정보 인증요청
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
 
# 엑세스 요청
auth.set_access_token(access_token, access_token_secret)
 
# twitter api 생성
api = tweepy.API(auth) # , wait_on_rate_limit_notify=True, wait_on_rate_limit=True

In [3]:
def process_results(results):
    id_list = [tweet.id for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])

    data_set["text"] = [(tweet.retweeted_status.full_text if hasattr(tweet, 'retweeted_status') else tweet.full_text) for tweet in results]    
    #data_set["text"] = [tweet.full_text for tweet in results]    
    data_set["hashtags"] = [tweet.entities['hashtags'] for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]

    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]

    return data_set

In [4]:
keyword = 'pfizer' # 검색할 키워드 입력
result = [] # 게시글을 저장할 리스트 변수

cursor = tweepy.Cursor(api.search, 
                       q=keyword,
                       since='2021-10-13', # 이곳은 오늘로부터 7일 전부터만 가능함. (오늘이 2021-10-12라면 2021-10-05부터 가능) 
                       until='2021-10-19', # 이곳은 since 날짜의 +1을 하세요 (ex. since=2021-10-05라면 until=2021-10-06으로 설정)
                       include_entities=True,
                       count=180,
                       lang="en",
                       tweet_mode='extended').items()
while True:
    try:
        tweet = cursor.next()
        result.append(tweet)
    except tweepy.TweepError: # request limit에 도달할때마다 저장
        data_set = process_results(result) 
        data_set.to_excel('./dataset/pfizer1013-1019.xlsx', index=False)     
        
        print('sleep 15min..')
        for i in range(15): # 15번 반복
            time.sleep(60 * 1) # 1분 휴식        
        print('break')
        
        continue
    except StopIteration:
        break
    except SleepInterruptedException:
        break

sleep 15min..
break
sleep 15min..
break
sleep 15min..
break
sleep 15min..
break
sleep 15min..
break
sleep 15min..
break


In [5]:
data_set = process_results(result)

In [6]:
data_set.to_excel('./dataset/pfizer1013-1019.xlsx', index=False) # 주피터노트북 끄기 전에 반드시 저장을 따로 실행하고 저장됬는지 확인